# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    #tick_act = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        #tick_act.append(tick_list[-1])
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
    #temp_df['tick_act'] = tick_act      
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

#### 1. High number of ticks ensures pip level movement and pip level predictions
#### 2. High sma_len helps ml model prediction accuracy (Predictable curve)
#### 3. High sma_len remove connection between actual tick_avg and sema (Prediction is high, but actual prediction is poor)

In [6]:
year = 2019

data = {}
data['number_of_ticks'] = 300
data['rsi_window'] = 14
data['sma_len'] = 5
data['lma_len'] = 10

diff_col = 'sema'
data['pip_diff'] = 0.0001

source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [7]:
%%time
df = pd.read_csv(source_file_path, nrows=10000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 6.06 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████████████████████████████████████████████████| 33334/33334 [00:10<00:00, 3214.63it/s]


Records : 33334
Wall time: 11.2 s


,tick_avg,spread_avg,tick_sd
0,1.146473,0.000481,0.000160
1,1.146313,0.000478,0.000066
2,1.146300,0.000327,0.000062
3,1.146311,0.000254,0.000030
4,1.146350,0.000418,0.000063


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 38.9 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

33330it [00:31, 1046.87it/s]
33325it [00:31, 1052.20it/s]

Wall time: 1min 3s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

33326it [00:09, 3489.01it/s]
33321it [00:09, 3474.90it/s]

Wall time: 19.1 s


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 33321
Wall time: 35.9 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
33316,1.124068,0.000029,0.000087,1.123907,1.123996,0.000091,0.000006,0.000302,0.000065,0.000077,...,-2.877500e-05,-0.000254,1.124068,1.123727,0.000000,-0.000341,-75.269034,-83.083159,-8.958998e-05,increase
33317,1.124171,0.000028,0.000078,1.124017,1.124018,0.000111,0.000022,0.000103,0.000062,0.000077,...,-3.601500e-05,-0.000129,1.124171,1.123766,0.000000,-0.000406,60.014450,-82.252419,-5.998274e-07,increase
33318,1.124375,0.000028,0.000033,1.124163,1.124078,0.000146,0.000060,0.000204,0.000076,0.000075,...,-1.814667e-05,-0.000007,1.124375,1.123766,0.000000,-0.000609,82.363138,-78.251664,8.525210e-05,increase
33319,1.124471,0.000027,0.000027,1.124296,1.124152,0.000133,0.000074,0.000096,0.000083,0.000069,...,8.093333e-06,0.000105,1.124471,1.123766,0.000000,-0.000706,85.299407,-53.083862,1.444227e-04,same
33320,1.124408,0.000027,0.000049,1.124366,1.124198,0.000070,0.000046,-0.000063,0.000077,0.000073,...,-8.783333e-07,0.000234,1.124471,1.124068,0.000063,-0.000341,85.224854,69.649012,1.679490e-04,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 1.25 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       26373   79.148285
decrease    3602   10.810000
increase    3346   10.041715


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.146674,0.000066,0.000033,1.146645,1.146574,0.000022,0.000035,0.000021,0.000045,0.000030,...,0.000036,0.000133,1.146674,1.146587,0.000000,-0.000087,72.840045,78.091169,0.000071,same
1,1.146695,0.000064,0.000047,1.146665,1.146606,0.000020,0.000032,0.000021,0.000046,0.000030,...,0.000035,0.000112,1.146695,1.146587,0.000000,-0.000108,68.684905,78.874888,0.000059,same
2,1.146682,0.000056,0.000017,1.146675,1.146635,0.000011,0.000029,-0.000013,0.000046,0.000020,...,0.000054,0.000077,1.146695,1.146613,0.000013,-0.000069,63.352971,77.576158,0.000041,same
3,1.146594,0.000053,0.000024,1.146648,1.146637,-0.000028,0.000002,-0.000088,0.000046,0.000025,...,0.000034,0.000039,1.146695,1.146594,0.000101,0.000000,38.347811,73.138665,0.000010,same
4,1.146523,0.000050,0.000051,1.146599,1.146617,-0.000048,-0.000020,-0.000071,0.000045,0.000030,...,0.000002,0.000011,1.146695,1.146523,0.000172,0.000000,-47.239966,62.713918,-0.000018,decrease


In [15]:
# Direction -------------------------
#diff_col = 'tick_act'
diff_col = 'tick_avg'

df['act_direction'] = 'same'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] > 0.0001] = 'increase'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -0.0001] = 'decrease'

print('prediction : same')
print(df.loc[df['direction'] == 'same', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : increase')
print(df.loc[df['direction'] == 'increase', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : decrease')
print(df.loc[df['direction'] == 'decrease', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('\n')
print(df[['tick_avg', 'sema', 'direction','act_direction']].head(50))
del df['act_direction']

prediction : same
same        0.646495
increase    0.179919
decrease    0.173587
Name: act_direction, dtype: float64
-------------
prediction : increase
increase    0.781829
same        0.204722
decrease    0.013449
Name: act_direction, dtype: float64
-------------
prediction : decrease
decrease    0.792615
same        0.189062
increase    0.018323
Name: act_direction, dtype: float64
-------------


    tick_avg      sema direction act_direction
0   1.146674  1.146645      same          same
1   1.146695  1.146665      same          same
2   1.146682  1.146675      same          same
3   1.146594  1.146648      same          same
4   1.146523  1.146599  decrease      decrease
5   1.146239  1.146457  decrease          same
6   1.146155  1.146329  decrease      decrease
7   1.145950  1.146166      same      increase
8   1.146061  1.146104      same          same
9   1.146097  1.146080      same          same
10  1.146157  1.146102      same          same
11  1.146084  1.146092      same 